In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy import signal
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('seaducks'), '..')))
from shapely import points,Polygon
from seaducks.utils import identify_time_series_segments

In [2]:
# load drifter dataset
drifter_dataset = pd.read_hdf('../data/corrected_velocity_drifter_full.h5')

In [3]:
df = drifter_dataset.copy()

In [4]:
df.loc[:,'segment_id'] = df[['id','time']].groupby('id')['time'].transform(identify_time_series_segments)

In [7]:
df.shape[0]

37053726

In [5]:
variables = list(df.columns)
grouped_df = df.groupby(['id', 'segment_id'])[variables]

In [11]:
grouped_df.filter(lambda x:len(x)>30).groupby(['id', 'segment_id'])[variables].apply(lambda x:x)

lon        lat        id  \
id       segment_id                                            
1831     0          6003879  -88.901001  -9.957000      1831   
                    6004892  -88.952003  -9.947000      1831   
                    6005541  -89.004997  -9.921000      1831   
                    6006036  -89.067001  -9.918000      1831   
                    6007029  -89.125999  -9.932000      1831   
...                                 ...        ...       ...   
68310100 0          37047466  95.125999 -32.094002  68310100   
                    37048882  95.084999 -32.113998  68310100   
                    37050319  95.089996 -32.132999  68310100   
                    37051601  95.093002 -32.109001  68310100   
                    37052690  95.053001 -32.106998  68310100   

                                            time  drogue          u       v  \
id       segment_id                                                           
1831     0          6003879  2000-03-07 00:00:00    True -23.565001   6.068   
                    6004892  2000-03-07 06:00:00    True -26.355000   9.219   
                    6005541  2000-03-07 12:00:00    True -29.219999   7.344   
                    6006036  2000-03-07 18:00:00    True -30.730000  -2.773   
                    6007029  2000-03-08 00:00:00    True -28.751999  -8.171   
...                                          ...     ...        ...     ...   
68310100 0          37047466 2019-12-30 18:00:00    True -14.271000   1.026   
                    37048882 2019-12-31 00:00:00    True  -7.795000 -10.212   
                    37050319 2019-12-31 06:00:00    True   1.627000   1.271   
                    37051601 2019-12-31 12:00:00    True  -8.175000   6.889   
                    37052690 2019-12-31 18:00:00    True -14.635000  -4.009   

                                    Wx        Wy        Tx  ...      u_av  \
id       segment_id                                         ...             
1831     0          6003879  -4.907712  5.124260 -0.047464  ... -0.126644   
                    6004892  -4.568380  4.614385 -0.035198  ...       NaN   
                    6005541  -4.844738  5.412632 -0.067978  ...       NaN   
                    6006036  -5.537863  4.025917 -0.054198  ...       NaN   
                    6007029  -4.712149  5.815896 -0.053733  ... -0.138294   
...                                ...       ...       ...  ...       ...   
68310100 0          37047466  1.810533  1.871846  0.005269  ...       NaN   
                    37048882  2.371617  2.030496  0.012564  ... -0.009495   
                    37050319  2.918461  3.999624  0.020309  ...       NaN   
                    37051601  1.993117  4.335700  0.014492  ...       NaN   
                    37052690  1.464187  2.947490  0.007482  ...       NaN   

                                  v_av       adt       sla      ugos  \
id       segment_id                                                    
1831     0          6003879   0.003325  0.539368 -0.004751 -0.139657   
                    6004892        NaN       NaN       NaN       NaN   
                    6005541        NaN       NaN       NaN       NaN   
                    6006036        NaN       NaN       NaN       NaN   
                    6007029   0.018790  0.539661 -0.004622 -0.153995   
...                                ...       ...       ...       ...   
68310100 0          37047466       NaN       NaN       NaN       NaN   
                    37048882  0.072860  0.640388  0.060248  0.017378   
                    37050319       NaN       NaN       NaN       NaN   
                    37051601       NaN       NaN       NaN       NaN   
                    37052690       NaN       NaN       NaN       NaN   

                                  vgos   adt_err   lon_var   lat_var  \
id       segment_id                                                    
1831     0          6003879   0.009523  0.007861  0.000028  0.000028   
                    

In [6]:
lat = df['lat'].values
time_dependent_vars = ['u','v','Wx','Wy','Tx','Ty']

In [8]:
# prevent changes to the data outside of this function
for var in time_dependent_vars:
    if var + '_filtered' not in df.columns:
        df[var + '_filtered'] = df[var].copy()
vars_to_filter = [var + '_filtered' for var in time_dependent_vars]

In [10]:
time_series = df[vars_to_filter].values

In [32]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',

                          'foo', 'bar','bar'],

                   'B' : [1, 2, 3, 4, 5, 6,7],

                   'C' : [2.0, 5., 8., 1., 2., 9.,10.]})

grouped = df.groupby('A')

In [34]:
grouped.filter(lambda x:len(x)>3)

,A,B,C
1,bar,2,5.0
3,bar,4,1.0
5,bar,6,9.0
6,bar,7,10.0
